In [ ]:
####

In [ ]:
import torch
from torch import nn
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchvision import transforms
from tqdm.notebook import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def Reverse(lst):
    return [ele for ele in reversed(lst)]

In [ ]:
def show_tensor_images(image_tensor, num_images=2, size=(3 , 416 , 416)):
    image_shifted = image_tensor
    image_unflat = image_shifted.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
def iou_width_height(boxes1, boxes2):

    intersection = torch.min(boxes1[..., 0], boxes2[..., 0]) * torch.min(
        boxes1[..., 1], boxes2[..., 1]
    )
    union = (
        boxes1[..., 0] * boxes1[..., 1] + boxes2[..., 0] * boxes2[..., 1] - intersection
    )
    return intersection / union

In [ ]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [ ]:

def iou(bboxes1 , bboxes2):
    area1 = (bboxes1[..., 2] - bboxes1[..., 0]) * (bboxes1[..., 3] - bboxes1[..., 1])
    area2 = (bboxes2[..., 2] - bboxes2[..., 0]) * (bboxes2[..., 3] - bboxes2[..., 1])
    inter_max_xy = torch.min(bboxes1[..., 2:],bboxes2[..., 2:])
    inter_min_xy = torch.max(bboxes1[..., :2],bboxes2[..., :2])
    #print(inter_max_xy.shape , inter_min_xy.shape)
    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[..., 0] * inter[..., 1]
    union = area1+area2-inter_area
    ious = inter_area / union
    ious = torch.clamp(ious,min=0,max = 1.0)
    return ious


def giou(bboxes1 , bboxes2):
    area1 = (bboxes1[..., 2] - bboxes1[..., 0]) * (bboxes1[..., 3] - bboxes1[..., 1])
    area2 = (bboxes2[..., 2] - bboxes2[..., 0]) * (bboxes2[..., 3] - bboxes2[..., 1])
    inter_max_xy = torch.min(bboxes1[..., 2:],bboxes2[..., 2:])
    inter_min_xy = torch.max(bboxes1[..., :2],bboxes2[..., :2])
    out_max_xy = torch.max(bboxes1[..., 2:],bboxes2[..., 2:])
    out_min_xy = torch.min(bboxes1[..., :2],bboxes2[..., :2])
    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[..., 0] * inter[..., 1]
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_area = outer[..., 0] * outer[..., 1]
    union = area1+area2-inter_area
    closure = outer_area
    ious = inter_area / union - (closure - union) / closure
    ious = torch.clamp(ious,min=-1.0,max = 1.0)
    return ious

def diou(bboxes1 , bboxes2):
    w1 = bboxes1[..., 2] - bboxes1[..., 0]
    h1 = bboxes1[..., 3] - bboxes1[..., 1]
    w2 = bboxes2[..., 2] - bboxes2[..., 0]
    h2 = bboxes2[..., 3] - bboxes2[..., 1]

    area1 = w1 * h1
    area2 = w2 * h2
    center_x1 = (bboxes1[..., 2] + bboxes1[..., 0]) / 2
    center_y1 = (bboxes1[..., 3] + bboxes1[..., 1]) / 2
    center_x2 = (bboxes2[..., 2] + bboxes2[..., 0]) / 2
    center_y2 = (bboxes2[..., 3] + bboxes2[..., 1]) / 2

    inter_max_xy = torch.min(bboxes1[..., 2:],bboxes2[..., 2:])
    inter_min_xy = torch.max(bboxes1[..., :2],bboxes2[..., :2])
    out_max_xy = torch.max(bboxes1[..., 2:],bboxes2[..., 2:])
    out_min_xy = torch.min(bboxes1[..., :2],bboxes2[..., :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[..., 0] * inter[..., 1]
    inter_diag = (center_x2 - center_x1)**2 + (center_y2 - center_y1)**2
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_diag = (outer[..., 0] ** 2) + (outer[..., 1] ** 2)
    union = area1+area2-inter_area
    dious = inter_area / union - (inter_diag) / outer_diag
    dious = torch.clamp(dious,min=-1.0,max = 1.0)
    return dious

def ciou(bboxes1 , bboxes2):
    w1 = bboxes1[..., 2] - bboxes1[..., 0]
    h1 = bboxes1[..., 3] - bboxes1[..., 1]
    w2 = bboxes2[..., 2] - bboxes2[..., 0]
    h2 = bboxes2[..., 3] - bboxes2[..., 1]

    area1 = w1 * h1
    area2 = w2 * h2

    center_x1 = (bboxes1[..., 2] + bboxes1[..., 0]) / 2
    center_y1 = (bboxes1[..., 3] + bboxes1[..., 1]) / 2
    center_x2 = (bboxes2[..., 2] + bboxes2[..., 0]) / 2
    center_y2 = (bboxes2[..., 3] + bboxes2[..., 1]) / 2

    inter_max_xy = torch.min(bboxes1[..., 2:],bboxes2[..., 2:])
    inter_min_xy = torch.max(bboxes1[..., :2],bboxes2[..., :2])
    out_max_xy = torch.max(bboxes1[..., 2:],bboxes2[..., 2:])
    out_min_xy = torch.min(bboxes1[..., :2],bboxes2[..., :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[..., 0] * inter[..., 1]
    inter_diag = (center_x2 - center_x1)**2 + (center_y2 - center_y1)**2
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_diag = (outer[..., 0] ** 2) + (outer[..., 1] ** 2)
    union = area1+area2-inter_area
    u = (inter_diag) / outer_diag
    iou = inter_area / union
    v = (4 / (math.pi ** 2)) * torch.pow((torch.atan(w2 / h2) - torch.atan(w1 / h1)), 2)
    with torch.no_grad():
        S = 1 - iou
        alpha = v / (S + v)
    cious = iou - (u + alpha * v)
    cious = torch.clamp(cious,min=-1.0,max = 1.0)
    return cious



In [ ]:
class Conv(nn.Module):
    def __init__(self ,
                 in_channels , 
                 out_channels , 
                 kernel_size = (3 , 3) , 
                 stride = (1 , 1) , 
                 padding = 1 , 
                 use_norm = True , 
                 use_activation = True , 
                 use_pooling = False):
        super(Conv , self).__init__()

        self.use_norm = use_norm
        self.use_activation = use_activation
        self.use_pooling = use_pooling

        self.conv1 = nn.Conv2d(in_channels , 
                               out_channels ,
                               kernel_size , 
                               stride , 
                               padding)
        if self.use_norm:
            self.norm = nn.InstanceNorm2d(out_channels)
        if self.use_activation:
            self.activation = nn.Mish()

        if self.use_pooling:
            self.pooling = nn.MaxPool2d(kernel_size=(2 , 2) , stride=(2 , 2))

    def forward(self , x):

        x = self.conv1(x)
        if self.use_norm:
            x = self.norm(x)
        if self.use_activation:
            x = self.activation(x)
        if self.use_pooling:
            x = self.pooling(x)
        return x

In [ ]:
class ConvT(nn.Module):
    def __init__(self , 
                 in_channels , 
                 out_channels , 
                 kernel_size = (2 , 2) , 
                 stride = (2 , 2) , 
                 padding = 0 , 
                 use_norm = True , 
                 use_activation = True):
        super(ConvT , self).__init__()

        self.use_norm = use_norm
        self.use_activation = use_activation

        self.convT = nn.ConvTranspose2d(in_channels , 
                                        out_channels , 
                                        kernel_size , 
                                        stride ,
                                        padding)
        if self.use_norm:
            self.norm = nn.InstanceNorm2d(out_channels)
        if self.use_activation:
            self.activation = nn.Mish()

    def forward(self , x):
        x = self.convT(x)
        if self.use_norm:
            x = self.norm(x)
        if self.use_activation:
            x = self.activation(x)
        return x

In [ ]:
class Darknet_Conv(nn.Module):
    def __init__(self ,
                 in_channels , 
                 out_channels):
        super(Darknet_Conv , self).__init__()

        self.conv1 = Conv(in_channels , 
                          in_channels , 
                          kernel_size = (1 , 1) , 
                          stride = (1 , 1) , 
                          padding = 0)
        self.conv2 = Conv(in_channels , in_channels)
        self.conv3 = Conv(in_channels , 
                          out_channels , 
                          kernel_size=(1 , 1) , 
                          stride = (1 ,1 ) , 
                          padding = 0)

    def forward(self , x):
        x_ = x.clone()
        x = self.conv1(x)
        x = self.conv2(x)
        x += x_
        x = self.conv3(x)
        return x

In [ ]:
class CAM(nn.Module):
    ## Channel Attention Module
    def __init__(self , 
                 in_channels , 
                 r=1):
        super(CAM , self).__init__()

        self.adp_max_pool = nn.AdaptiveMaxPool2d(output_size=(1 , 1))
        self.adp_avg_pool = nn.AdaptiveAvgPool2d(output_size=(1 , 1))
        self.linear = nn.Linear(in_channels , in_channels//r)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self , x):
        x_max = self.adp_max_pool(x)
        x_avg = self.adp_avg_pool(x)
        #print(x_max.shape , x_avg.shape)
        x_max = self.linear(x_max.squeeze(-1).squeeze(-1))
        x_avg = self.linear(x_avg.squeeze(-1).squeeze(-1))

        x = x_max + x_avg
        x = self.sigmoid(x)
        return x.view(x.shape[0] , x.shape[1] , 1 , 1)

In [ ]:
class SAM(nn.Module):
    ## Spatial Attention Module
    def __init__(self , 
                 in_channels):
        super(SAM , self).__init__()

        self.conv1 = Conv(in_channels , 1 , kernel_size=(1 , 1) , stride=(1 , 1) , padding=0)

        self.conv2 = Conv(1 , in_channels//2)
        self.conv3 = Conv(in_channels//2 , 1)
    def forward(self , x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return x

In [ ]:
class CBAM(nn.Module):
    def __init__(self , 
                 in_channels):
        super(CBAM , self).__init__()

        self.sam = SAM(in_channels)
        self.cam = CAM(in_channels)

    def forward(self , x):
        x_ = x.clone()
        x = self.cam(x)
        x = x_ * x
        x = self.sam(x)
        x = x_ * x
        return x

In [ ]:
class DenseNet_Block(nn.Module):
    def __init__(self , 
                 in_channels , 
                 repeats , 
                 k = 12):
        super(DenseNet_Block , self).__init__()

        self.layers = nn.ModuleList()
        out_channels_last = 0
        in_channels_conv = in_channels // 3
        self.remaining_in_channels = in_channels - in_channels_conv

        self.in_channels_ = in_channels
        self.in_channels_conv_ = in_channels_conv
        for r in range(repeats):
            #in_channels = in_channels + out_channels_last
            out_channels = in_channels_conv + k 
            #print(in_channels , out_channels)
            self.layers.append(Darknet_Conv(in_channels_conv , 
                                             out_channels))
            self.layers.append(CBAM(out_channels))
            
            out_channels_last += in_channels_conv
            in_channels_conv = out_channels + out_channels_last
        #print(out_channels)
        self.out_channels = out_channels + self.remaining_in_channels+ out_channels_last

        
    def _concat(self , x , prev):
        for prev_ in prev:
            x = torch.cat([x , prev_] , dim=1)
        return x

    def forward(self , x):
        x_cpy = x.clone()
        x = x[: , :self.in_channels_conv_ , : , :]
        x_ = x_cpy[: , self.in_channels_conv_ : self.in_channels_ , : , :]
        #print(x_.shape)
        prev = [x]
        for layer in self.layers:
            #print(x.shape)
            x = layer(x)
            x = self._concat(x , prev)
            prev.append(x)
        #print(x.shape , x_.shape)
        x = torch.cat([x , x_] , dim=1)
        return x

In [ ]:
config = [
          [64 , (3 , 3) , (1 , 1) , 1 , True , True , True] ,  # [out_channels , kernel_size , stride , padding , norm , activation , pooling]
          1 , 
          [128 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          2 , 
          [256 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          8 , 
          [512 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          8 , 
          [1024 , (3 , 3) , (1 , 1) , 1 , True , True , True] , 
          4
]

In [ ]:
class CSPDarknet(nn.Module):
    def __init__(self , 
                 in_channels , 
                 config = config):
        super(CSPDarknet , self).__init__()

        self.layers = nn.ModuleList()
        out_channels_list = [64 , 128 , 256 , 512 , 1024]
        i = 0
        for  layer in config:
            if isinstance(layer , list):
                out_channels , kernel_size , stride , padding , use_norm , use_activation , use_padding = layer
                self.layers.append(Conv(in_channels , out_channels , kernel_size , stride , padding , use_norm , use_activation , use_padding))
                in_channels = out_channels

            elif isinstance(layer , int):
                repeats = layer
                a = DenseNet_Block(in_channels , repeats)
                self.layers.append(a)
                in_channels = a.out_channels
                out_channels = out_channels_list[i]
                self.layers.append(Conv(in_channels , out_channels , kernel_size=(1 , 1) , stride=(1 , 1) , padding=0))
                i += 1
                in_channels = out_channels

                
    def forward(self , x):
        layer_out = []
        for i , layer in enumerate(self.layers):
            x = layer(x)
            if i != 0 :
                layer_out.append(x)
        return layer_out

In [ ]:
'''cspdensenet = CSPDarknet(3).to(device)
x = torch.randn(2 , 3 , 416 , 416).to(device)
z = cspdensenet(x)
for data in z:
    print(data.shape)'''

'cspdensenet = CSPDarknet(3).to(device)\nx = torch.randn(2 , 3 , 416 , 416).to(device)\nz = cspdensenet(x)\nfor data in z:\n    print(data.shape)'

In [ ]:
class PAN_Net(nn.Module):
    def __init__(self , 
                 in_channels = [2048 , 1024 , 512 , 256] ,
                 out_channels = [128 , 256 , 512 , 1024]):
        super(PAN_Net , self).__init__()

        self.top_down = nn.ModuleList()
        self.bottom_up = nn.ModuleList()
        
        for i , channel in enumerate(in_channels):
            if i > 2 and i != len(in_channels) -1:
                channel = channel + in_channels[i-1]
            out_channel = channel // 2
            self.top_down.append(ConvT(channel , out_channel))
            #channel = out_channel
        
        for i , channel in enumerate(out_channels):
            if i > 2 and i!= len(out_channels)-1 :
                channel = channel * 2 
            out_channel = out_channels[i] * 2
            self.bottom_up.append(Conv(channel , out_channel , use_pool=True))

        #print(self.bottom_up)
    def forward(self , x):
        p = [x[0]]
        N = []
        x0 , x1 , x2 , x3 = x
        for i , layer in enumerate(self.top_down):
            if i > 2 and i!= len(self.top_down)-1:
                #p[i] = p[i] + x[i+1]
                #print(i , len(p) , len(x))
                p[i] = torch.cat([p[i] , x[i]] , dim=1)
            #print(p[i].shape)
            p.append(layer(p[i]))
            #print(p[i].shape)

        p_ = Reverse(p)
        N.append(p_[0])
        for i , layer in enumerate(self.bottom_up):
            if i > 2 and i!= len(self.top_down)-1:
                #N[i] = N[i] + p_[i+1]
                N[i] = torch.cat([N[i] , p_[i]] , dim=1)
            #print(N[i].shape)
            N.append(layer(N[i]))
            #print(N[i].shape)
        return N[1:]

In [ ]:
class Residual_Block(nn.Module):
    def __init__(self , 
                 in_channels , 
                 num_repeats , 
                 residual = True):
        super(Residual_Block , self).__init__()

        self.residual = residual
        self.num_repeats = num_repeats
        layers = []


        for repeat in range(self.num_repeats):
            layers.append(Conv(
                in_channels , 
                in_channels // 2 ,
                kernel_size = (1 , 1) , 
                stride = (1 , 1) , 
                padding = 0
            ))

            layers.append(Conv(
                in_channels //2 , 
                in_channels
            ))

        self.conv = nn.Sequential(*layers)
    def forward(self , x):
        if self.residual:
            x_ = self.conv(x)
            x = x + x_
            return x
        else:
            x = self.conv(x)
            return x

In [ ]:
class Scaled_Predictions(nn.Module):
    def __init__(self , 
                 in_channels , 
                 out_channels , 
                 B = 3):
        super(Scaled_Predictions , self).__init__()
        self.B = B
        self.out_channels = out_channels
        self.conv1 = Conv(in_channels , in_channels * 2 )
        self.conv2 = Conv(in_channels * 2 , (self.out_channels + 5) * B , use_norm=False , use_activation=False)

    def forward(self , x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.shape[0] , self.B , x.shape[2] , x.shape[3] , self.out_channels + 5 )
        return x

In [ ]:
class YOLO_v3(nn.Module):
    def __init__(self , 
                 in_channels , 
                 out_channels = 20):
        super(YOLO_v3 , self).__init__()

        self.residual_block = Residual_Block(in_channels , 1 , False)
        self.conv1 = Conv(in_channels , in_channels // 2 , kernel_size=(1 , 1) , stride=(1 , 1) , padding=0)
        self.scaled_pred = Scaled_Predictions(in_channels//2 , out_channels)

    def forward(self , x):
        x = self.residual_block(x)
        x = self.conv1(x)
        x = self.scaled_pred(x)
        return x

In [ ]:
class YOLO_v4(nn.Module):
    def __init__(self , 
                 in_channels = 3):
        super(YOLO_v4 , self).__init__()

        out_channels_list = [128 , 256 , 512 , 1024]

        self.yolo_v3_layers = nn.ModuleList()
        self.csp_darknet = CSPDarknet(in_channels)
        self.pannet = PAN_Net()
        
        for i in out_channels_list:
            self.yolo_v3_layers.append(
                YOLO_v3(i)
            )
    
    def forward(self , x):
        out = []
        x = self.csp_darknet(x)
        x = self.pannet(x)

        for i , x_ in enumerate(x):
            y = self.yolo_v3_layers[i](x_)
            out.append(y)
        return out

In [ ]:
anchors = [
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)],
]

In [ ]:
class Dataset_(torch.utils.data.Dataset):
    def __init__(
        self,
        csv_file,
        img_dir,
        label_dir,
        anchors = anchors,
        S=[13, 26, 52 , 104],
        C=20,
        transform=None,
    ):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.S = S
        self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2])
        self.num_anchors = self.anchors.shape[0]
        self.num_anchors_per_scale = self.num_anchors // 3
        self.C = C
        self.ignore_iou_thresh = 0.5

    def __len__(self):
        return len(self.df)

    def __getitem__(self , idx):
        label_path = os.path.join(self.label_dir , self.df.iloc[idx , 1])
        boxes = []

        with open(label_path) as f:
            for label in f.readlines():
                class_label , x , y , width , height = [
                    float(x) if float(x) != int(float(x)) else int(x)
                    for x in label.replace("\n", "").split()
                ]
                boxes.append([ x , y , width , height , class_label])

        boxes = torch.tensor(boxes) 

        image_path = os.path.join(self.img_dir , self.df.iloc[idx , 0])
        image = np.asarray(plt.imread(image_path))
        image = torch.from_numpy(image).permute(2 , 0 , 1)

        if self.transform:
            image = self.transform(image)
        
        target = [torch.zeros((self.num_anchors // 3 , S , S , 25)) for S in self.S]

        for box in boxes:
            iou_anchors = iou_width_height(box[2:4] , self.anchors)
            anchors_indices = iou_anchors.argsort(descending = True , dim = 0)

            x , y , width , height , class_label = box
            has_anchor = [False] * 3
            for anchor_idx in anchors_indices:
                scale_idx = anchor_idx // self.num_anchors_per_scale
                anchor_on_scale = anchor_idx % self.num_anchors_per_scale
                S = self.S[scale_idx]
                i , j = int(S * y) , int(S * x)
                anchor_taken = target[scale_idx][anchor_on_scale , i , j , 0]
                if not anchor_taken and not has_anchor[scale_idx] :
                    target[scale_idx][anchor_on_scale, i , j , 0] = 1
                    x_cell , y_cell = S * x - j , S * y - i
                    width_cell , height_cell = (
                        width * S , 
                        height * S
                    )
                    box_coordinates = torch.tensor(
                        [x_cell , y_cell , width_cell , height_cell]
                    )
                    target[scale_idx][anchor_on_scale , i , j , 1:5] = box_coordinates
                    target[scale_idx][anchor_on_scale , i , j , 5] = int(class_label)
                    has_anchor[scale_idx] = True

                elif not anchor_taken and iou_anchors[anchor_idx] > self.ignore_iou_thresh:
                    target[scale_idx][anchor_on_scale , i , j , 0] = -1
        return image , tuple(target)

In [ ]:
transform = transforms.Compose([
                                transforms.ToPILImage() , 
                                transforms.Resize((416 , 416)) , 
                                transforms.ToTensor()
])
dataset = Dataset_(
    img_dir = '/content/drive/MyDrive/Yolo_Dataset/images/' , 
    label_dir = '/content/drive/MyDrive/Yolo_Dataset/labels' , 
    csv_file = '/content/drive/MyDrive/Yolo_Dataset/train.csv' , 
    transform = transform
)
dataloader = torch.utils.data.DataLoader(dataset , batch_size = 1 , shuffle=True)

In [ ]:
for x , y in dataloader:
    show_tensor_images(x)
    for data in y:
        print(data.shape)

In [ ]:
class IOU_Loss(nn.Module):
    def __init__(self , 
                 loss_type = 'iou'): ## loss types are [iou , giou , diou , ciou]
        super(IOU_Loss , self).__init__()

        self.loss_type = loss_type

    def forward(self , pred_box , trg_box):
        if self.loss_type == 'iou':
            loss = torch.sum(1.0 - iou(pred_box , trg_box))
        else :
            if self.loss_type == 'giou':
                loss = torch.sum(1.0 - giou(pred_box , trg_box))
            else:
                if self.loss_type == 'diou':
                    loss = torch.sum(1.0 - diou(pred_box , trg_box))
                else:
                    loss = torch.sum(1.0 - ciou(pred_box , trg_box))
        return loss

In [ ]:
yolo_v4 = YOLO_v4().to(device)
lr = 0.002
betas = (0.5 , 0.999)
opt = torch.optim.Adam(yolo.parameters() , lr=lr , betas = betas)
epochs = 200
display_steps = 1
l1_criterion = nn.L1Loss()
loss_fn = IOU_Loss()

In [ ]:
def train():
    mean_yolo_loss = 0
    cur_step = 0

    for epoch in range(epochs):
        for x , y in tqdm(dataloader):
            x = x.to(device)
            y0, y1, y2 , y3 = (
                y[0].to(device),
                y[1].to(device),
                y[2].to(device),
                y[3].to(device))

            opt.zero_grad()
            out = yolo_v4(x)
            loss = (
                loss_fn(out[0][... , :4].squeeze(0), y0[... , :4].squeeze(0))
                + loss_fn(out[1][... , :4].squeeze(0), y1[... , :4].squeeze(0))
                + loss_fn(out[2][... , :4].squeeze(0), y2[... , :4].squeeze(0))
                + loss_fn(out[3][... , :4].squeeze(0), y3[... , :4].squeeze(0))
            )
            loss_classes = (
                l1_criterion(out[0][... , 5:].squeeze(0) , y0[... , 5:].squeeze(0)
                             + l1_criterion(out[1][... , 5:].squeeze(0) , y1[... , 5:].squeeze(0))
                             + l1_criterion(out[2][... , 5:].squeeze(0) , y2[... , 5:].squeeze(0)))
                             + l1_criterion(out[3][... , 5:].squeeze(0) , y3[... , 5:].squeeze(0)))
            )
            loss += loss_classes
            loss.backward()
            opt.step()
            
            mean_yolo_loss += loss.item() / display_steps
            if cur_step % display_steps == 0:
                print(f'Epoch {epoch} , Step {cur_step} , Mean YOLO Loss {mean_yolo_loss}')
            cur_step +=1
        mean_yolo_loss = 0

In [ ]:
train()